### Engineering

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
def is_grad_enabled(input):
    return torch.is_grad_enabled() and input.requires_grad

In [ ]:
def _broadcast(input):
    return input.clone()

In [ ]:
def _reduce(input):
    world_size = torch.distributed.get_world_size()
    
    if world_size == 1:
        return input
    
    torch.distributed.all_reduce(input)
    
    return input

In [ ]:
class Broadcast(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return _broadcast(input)
    
    @staticmethod
    def backward(ctx, grad_output):
        return _reduce(grad_output)

In [ ]:
def broadcast_with_forward_and_backward(inputs):
    if is_grad_enabled(inputs):
        outputs = Broadcast.apply(inputs)
    else:
        outputs = _broadcast(inputs)
    
    return outputs

In [ ]:
world_size = torch.distributed.get_world_size()

In [ ]:
sorted_params = sorted(param_list, key=lambda x: x.numel(), reverse=True)

In [ ]:
numel_per_rank = [0 for _ in range(world_size)]

In [ ]:
param_per_rank = [[] for _ in range(world_size)]

In [ ]:
for param in sorted_params:
    next_device = numel_per_rank.index(min(numel_per_rank))
    
    param_per_rank[next_device].append(param)
    numel_per_rank[next_device] += param.numel()

In [ ]:
step 1: send
step 2: sender continues their execution
step 3: recever stops
step 4: if, then

In [ ]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embedding, embedding_dim):
        super().__init__()
        
        world_size = torch.distributed.get_world_size()
        self.num_embedding = num_embedding
        self.embedding_dim_per_partrition = embedding_dim // world_size
        self.embedding = nn.Embedding(
            num_embedding,
            embedding_dim=self.embedding_dim_per_partrition
        )
        self.vocab_start_idx, self.vocab_end_idx = self.get_vocab_range(
            self.embedding_dim_per_partrition
        )
    
    def get_vocab_range(self, embedding_dim_per_partrition):
        rank = torch.distributed.get_rank()
        start_idx = rank*embedding_dim_per_partrition
        end_idx = start_idx+embedding_dim_per_partrition
        return start_idx, end_idx

    def forward(self, tokens):
        mask = (self.vocab_start_idx < tokens) | (tokens >= self.vocab_end_idx)
        tokens = tokens - self.vocab_start_idx
        tokens[mask] = 0.
        
        embeddings = self.embedding(tokens)
        mask_idxs = torch.where(mask == 0)[1]
        embedding[mask_idxs] = 0.
        
        return embedding

In [ ]:
step 1: replicate the model
step 2: micro-batch
step 3: grad
step 4: average all the gradient

In [ ]:
register > cached sram > main memory dram > 

In [ ]:
step 1: partrition the params
step 2: 

In [ ]:
gradient, params, optimizer 

In [ ]:
import os

In [ ]:
class MPU:
    def __init__(self, master_addr, master_port, backend):
        if not torch.distributed.is_initialized():
            rank = os.getenv("RANK")
            world_size = os.getenv("WORLD_SIZE")
            os.environ["MASTER_ADDR"] = master_addr
            os.environ["MASTER_PORT"] = master_port
            
            device_count = torch.cuda.device_count()
            
            if device_count > 0:
                device = rank % device_count
                torch.cuda.set_device(device)
            
            torch.distributed.init_process_group(
                rank=rank,
                world_size=world_size,
                backend=backend
            )

In [ ]:
class ParallelMLP(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.dense_h_to_4h = ColumnParallelLinear(
            input_size=hidden_size,
            output_size=hidden_size*4
        )
        self.gelu = nn.GELU()
        self.dense_4h_to_h = RowParallelLinear(
            input_size=hidden_size*4,
            output_size=hidden_size
        )
    
    def forward(self, x):
        intermediate_activations = self.dense_h_to_4h(x)
        intermediate_activations = self.gelu(intermediate_activations)
        outputs = self.dense_4h_to_h(intermediate_activations)
        return outputs

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class CachedDataset(Dataset):
    def __init__(self, filename):
        self.filename = filename
        self.data = data
        self.cache_index = {}
    
    def prefetch(self, idxs):
        if all([x in self.cache_index for x in idxs]):
            return
        
        if not self.data:
            self.data = torch.load(self.filename)
        
        total_numbers = sum([self.data[x].numel() for x in idxs])
        
        self.cache = torch.randn(total_numbers, dtype=self.data.dtype)
        
        offset = 0
        for i in idxs:
            n_numbers = self.data[i].numel()
            self.cache[offset:offset+n_numbers]
            self.cache_index[i] = offset

In [ ]:
for i in range(num_tensor_model_parallel_groups):
    

In [ ]:
step 1: load data to main memory
step 2: the size
step 3: reserve
step 4: load


In [ ]:
shared memory, file system, message passing

In [ ]:
step 1: partrition
step 2: move
step 3: init local optimizer
step 4: sync locla optimizer
step 5: move local params to a bucket

### ML Engineering

In [ ]:
key, value, timestamp

In [ ]:
import threading

In [ ]:
thread = threading.Thread(target=print_numbers)

In [ ]:
thread.start()

In [ ]:
from typing import cast, List

In [ ]:
numbers = cast(List[int], numbers)

In [ ]:
step 1: create a network
step 2: attach the network
step 3:receive ip
step 4: communicate through the IPs

In [ ]:
from contextlib import contextmanager

In [ ]:
@contextmanager
def first_context():
    print("")
    

### Ai

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
def tokenize(x):
    return {
        "sentence1": tokenizer(x["tokenizer"])
    }

In [ ]:
tokenized_dataset = small_dataset.map(tokenize)

In [ ]:
def filter_func(x):
    return x["sentence1"].startswith("F")

In [ ]:
small_dataset.filter(filter_func)

In [ ]:
torch.distributed.broadcast(x, src=0, async_op=True)

In [ ]:
for param_group in optimizer.param_groups:
    for param in param_group["param"]:
        print(param.shape)

In [ ]:
def patch_func(activations, hook):
    activations[:, :, 4, :] = 0.
    return activations

In [ ]:
model.run_with_hooks(
    tokens,
    fwd_hooks=[(hook_name, patch_func)]
)

In [ ]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        world_size = torch.distributed.get_world_size()
        self.num_embeddings = num_embeddings
        self.embedding_dim_per_partrition = embedding_dim // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.num_embeddings,
            self.embedding_dim_per_partrition
        ))
        self.vocab_start_idx, self.vocab_end_idx = self.get_vocab_range(
            self.embedding_dim_per_partrition
        )
        
    def get_vocab_range(self, embedding_dim_per_partrition):
        rank = torch.distributed.get_rank()
        start_idx = rank*embedding_dim_per_partrition
        end_idx = start_idx+embedding_dim_per_partrition
        return start_idx, end_idx

    def forward(self, tokens):
        mask = (self.vocab_start_idx < tokens) | (tokens >= self.vocab_end_idx)
        mask_tokens = tokens - self.vocab_start_idx
        masked_tokens[mask] = 0.
        
        embeddings = F.embedding(masked_tokens, self.weight)
        mask_idxs = torch.where(mask == 0)[1]
        embeddings[mask_idxs] = 0.
        
        torch.distributed.all_reduce(embeddings)
        
        return embeddings

In [ ]:
from einops import rearrange

In [ ]:
output = rearrange(
    images,
    "b c (p_h n_h) (p_w n_w) -> b (n_h n_w) (p_h p_w c)"
)

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
logits = model(tokens)

In [ ]:
probs = F.softmax(logits, dim=-1)

In [ ]:
last_token_probs = probs[:, -1, :]

In [ ]:
target = tokens[1:]

In [ ]:
predicted_log_probs = -last_tokens_probs[target].log()

In [ ]:
repeated_tokens = model.to_tokens(repeated_text)

In [ ]:
induction_heads = [(6, 9), (4, 2)]

In [ ]:
attention_patterns = []

In [ ]:
_, cache = model.run_with_cache()

In [ ]:
for head_idx, layer_idx in induction_heads:
    

In [ ]:
text_embeddings = model.embed(tokens)

In [ ]:
positional_embeddings = model.pos_embed(tokens)

In [ ]:
embedding = text_embeddings + positional_embeddings

In [ ]:
residual = embedding

In [ ]:
for block in model.blocks:
    residual = block(residual)

In [ ]:
residual = model.ln_final(residual)

In [ ]:
logits = model.unembed(residual)

In [ ]:
task = torch.distributed.broadcast(x, src=0, async_op=True)

In [ ]:
task.wait()

In [ ]:
n_features = 5

In [ ]:
interference[torch.arange(n_features), torch.arange(n_features)] = 0

In [ ]:
polysemanticity = interference.pow(2).sum(dim=-1).sqrt() 

In [ ]:
cache["hook_embed"]

In [ ]:
cache["hook_pos_embd"]

In [ ]:
linear, residual connection, layernorm, multi, mask-multi,